In [1331]:
import pandas as pd
import numpy as np
import sqlite3

# Transforming the query result into dataframe for easier manipulations.
def query_to_df(query):
    with sqlite3.connect("sql-murder-mystery.db") as connection:
        results_df = pd.read_sql(query, connection)
    return results_df

# Retrieving the table schema
def get_schema(table_name):
    with sqlite3.connect("sql-murder-mystery.db") as connection:
        cursor = connection.cursor()
        cursor.execute(f"PRAGMA table_info('{table_name}')")
        data = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description] # retrieve column names
    df = pd.DataFrame(data, columns=columns) # create pandas DataFrame
    return df

# Checking what tables are in the database.
Firstly, I'm checking what tables are in the database file. <br> For that I'm gonna run a query on 'sqlite_master' table to receive information about the structure of a database.

In [1332]:
database_tables = query_to_df("SELECT name FROM sqlite_master WHERE type = 'table'")
print(database_tables)

                     name
0      crime_scene_report
1         drivers_license
2  facebook_event_checkin
3               interview
4      get_fit_now_member
5    get_fit_now_check_in
6                solution
7                  income
8                  person


# Schemas
Knowing what tables I'm working with, I'd check now what kind of data/information is stored inside each table.

### *Crime Scene Report*

In [1333]:
crime_scene_report_schema = get_schema("crime_scene_report")
print(crime_scene_report_schema)

   cid         name     type  notnull dflt_value  pk
0    0         date  INTEGER        0       None   0
1    1         type     TEXT        0       None   0
2    2  description     TEXT        0       None   0
3    3         city     TEXT        0       None   0


### *Drivers License*

In [1334]:
drivers_license_schema = get_schema("drivers_license")
print(drivers_license_schema)

   cid          name     type  notnull dflt_value  pk
0    0            id  INTEGER        0       None   1
1    1           age  INTEGER        0       None   0
2    2        height  INTEGER        0       None   0
3    3     eye_color     TEXT        0       None   0
4    4    hair_color     TEXT        0       None   0
5    5        gender     TEXT        0       None   0
6    6  plate_number     TEXT        0       None   0
7    7      car_make     TEXT        0       None   0
8    8     car_model     TEXT        0       None   0


### *Facebook Event Checkin*

In [1335]:
facebook_event_checkin_schema = get_schema("facebook_event_checkin")
print(facebook_event_checkin_schema)

   cid        name     type  notnull dflt_value  pk
0    0   person_id  INTEGER        0       None   0
1    1    event_id  INTEGER        0       None   0
2    2  event_name     TEXT        0       None   0
3    3        date  INTEGER        0       None   0


### *Interview*

In [1336]:
interview_schema = get_schema("interview")
print(interview_schema)

   cid        name     type  notnull dflt_value  pk
0    0   person_id  INTEGER        0       None   0
1    1  transcript     TEXT        0       None   0


### *Get Fit Now Member*

In [1337]:
get_fit_now_member_schema = get_schema("get_fit_now_member")
print(get_fit_now_member_schema)

   cid                   name     type  notnull dflt_value  pk
0    0                     id     TEXT        0       None   1
1    1              person_id  INTEGER        0       None   0
2    2                   name     TEXT        0       None   0
3    3  membership_start_date  INTEGER        0       None   0
4    4      membership_status     TEXT        0       None   0


### *Get Fit Now Check In*

In [1338]:
get_fit_now_check_in_schema = get_schema("get_fit_now_check_in")
print(get_fit_now_check_in_schema)

   cid            name     type  notnull dflt_value  pk
0    0   membership_id     TEXT        0       None   0
1    1   check_in_date  INTEGER        0       None   0
2    2   check_in_time  INTEGER        0       None   0
3    3  check_out_time  INTEGER        0       None   0


### *Solution*

In [1339]:
solution_schema = get_schema("solution_schema")
print(solution_schema)

Empty DataFrame
Columns: [cid, name, type, notnull, dflt_value, pk]
Index: []


### *Income*

In [1340]:
income_schema = get_schema("income")
print(income_schema)

   cid           name     type  notnull dflt_value  pk
0    0            ssn     CHAR        0       None   1
1    1  annual_income  INTEGER        0       None   0


### *Person*

In [1341]:
person_schema = get_schema("person")
print(person_schema)

   cid                 name     type  notnull dflt_value  pk
0    0                   id  INTEGER        0       None   1
1    1                 name     TEXT        0       None   0
2    2           license_id  INTEGER        0       None   0
3    3       address_number  INTEGER        0       None   0
4    4  address_street_name     TEXT        0       None   0
5    5                  ssn     CHAR        0       None   0


## First Clue:
A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it.
<br> You vaguely remember that the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018​ and that it took place in ​SQLCity​.

In [1342]:
first_clue_results = (query_to_df("""
                                    SELECT description
                                    FROM crime_scene_report
                                    WHERE date = 20180115 AND type = 'murder' AND city = 'SQL City'
                                  """))
with pd.option_context('display.max_colwidth', None):
    print(first_clue_results)

                                                                                                                                                                                 description
0  Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".


The results from the 1st clue:
- 1st. witness lives at the last house on "Northwestern Dr".
- 2nd. witness, named Annabel, lives somewhere on "Franklin Ave".
We can try to find more information about them in 'Person' table.

In [1343]:
x = (query_to_df("SELECT * FROM person"))
print(x)

          id                 name  license_id  address_number  \
0      10000   Christoper Peteuil      993845             624   
1      10007  Kourtney Calderwood      861794            2791   
2      10010            Muoi Cary      385336             741   
3      10016          Era Moselle      431897            1987   
4      10025         Trena Hornby      550890             276   
...      ...                  ...         ...             ...   
10006  99936          Luba Benser      274427             680   
10007  99941      Roxana Mckimley      975942            1613   
10008  99965      Cherie Zeimantz      287627            3661   
10009  99982          Allen Cruse      251350            3126   
10010  99990         Vance Hunten      830407            3056   

      address_street_name        ssn  
0            Bankhall Ave  747714076  
1           Gustavus Blvd  477972044  
2         Northwestern Dr  828638512  
3           Wood Glade St  614621061  
4           Daws Hill Wa

In [1344]:
second_clue_results = (query_to_df("""
                                    SELECT * 
                                    FROM person 
                                    WHERE address_street_name = 'Northwestern Dr' AND address_number = (SELECT MAX(address_number) FROM person WHERE address_street_name = 'Northwestern Dr')

                                    UNION

                                    SELECT * FROM person WHERE name LIKE '%Annabel%' AND address_street_name = 'Franklin Ave'
                                   """))
print(second_clue_results)

      id            name  license_id  address_number address_street_name  \
0  14887  Morty Schapiro      118009            4919     Northwestern Dr   
1  16371  Annabel Miller      490173             103        Franklin Ave   

         ssn  
0  111564949  
1  318771143  


Results from the 2nd clue:
- We know that the two witnesses are: Morty Schapiro (id: 14887) and Annabel Miller (id: 16371). Knowing that we can search for the transcripts in the Interview table.

In [1345]:
third_clue_results = (query_to_df("""
                                    SELECT interview.person_id, person.name, interview.transcript
                                    FROM interview
                                    JOIN person ON person.id = interview.person_id
                                    WHERE person.id IN (14887, 16371)
                                  """))

with pd.option_context('display.max_colwidth', None):
    print(third_clue_results)

   person_id            name  \
0      14887  Morty Schapiro   
1      16371  Annabel Miller   

                                                                                                                                                                                                                        transcript  
0  I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".  
1                                                                                                            I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.  


Results from the 3rd clue:
- Morty Schapiro saw a man with "Get Fit Now Gym" bag. Membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W"
- Annabel recognized the murderer from the gym. She saw the man last week while working out on January the 9th.


In [1346]:
info_from_morty = (query_to_df("""
                                SELECT * 
                                FROM get_fit_now_member 
                                WHERE membership_status = 'gold' AND id LIKE '48Z%'
                               """))
print(info_from_morty)
print('---------------------------------------------------------------------------')

info_about_drivers_license = (query_to_df("""
                                                      SELECT person.id, person.name, person.license_id
                                                      FROM person
                                                      JOIN get_fit_now_member ON person.id = get_fit_now_member.person_id
                                                      WHERE person.name IN ('Joe Germuska', 'Jeremy Bowers')
                                                     """))
print(info_about_drivers_license)

      id  person_id           name  membership_start_date membership_status
0  48Z7A      28819   Joe Germuska               20160305              gold
1  48Z55      67318  Jeremy Bowers               20160101              gold
---------------------------------------------------------------------------
      id           name  license_id
0  28819   Joe Germuska      173289
1  67318  Jeremy Bowers      423327


In [1347]:
drivers_license_info = (query_to_df("""
                                     SELECT *
                                     FROM drivers_license
                                     WHERE plate_number LIKE '%H42W%' AND id IN (173289, 423327)
                                    """))
print(drivers_license_info)

       id  age  height eye_color hair_color gender plate_number   car_make  \
0  423327   30      70     brown      brown   male       0H42W2  Chevrolet   

  car_model  
0  Spark LS  


In [1348]:
info_from_annabel = (query_to_df("SELECT * FROM get_fit_now_check_in WHERE check_in_date = '20180109'"))
print(info_from_annabel)
print('---------------------------------------------------------------------------')
info_about_annabel = (query_to_df("""
                                    SELECT *
                                    FROM get_fit_now_member
                                    JOIN get_fit_now_check_in ON get_fit_now_member.id = get_fit_now_check_in.membership_id
                                    WHERE get_fit_now_member.name LIKE '%Annabel%';
                                  """))
print(info_about_annabel)

  membership_id  check_in_date  check_in_time  check_out_time
0         X0643       20180109            957            1164
1         UK1F2       20180109            344             518
2         XTE42       20180109            486            1124
3         1AE2H       20180109            461             944
4         6LSTG       20180109            399             515
5         7MWHJ       20180109            273             885
6         GE5Q8       20180109            367             959
7         48Z7A       20180109           1600            1730
8         48Z55       20180109           1530            1700
9         90081       20180109           1600            1700
---------------------------------------------------------------------------
      id  person_id            name  membership_start_date membership_status  \
0  90081      16371  Annabel Miller               20160208              gold   

  membership_id  check_in_date  check_in_time  check_out_time  
0         90081  

Summing up all informations from the 4th clue:
1. Following Morty Schapiro transcript I searched for people with gold membership status whose bag started with "48Z". Then I checked their drivers license to get information about plate numbers. <br>
This returned the man named **Jeremy Bowers**.
2. Following Annabel Miller transcript I checked everyone that attempted the gym on 9th January 2018. I checked for the membership_id and check time, both in and out. <br>
**Jeremy Bowers** matches the description.

In [1349]:
jeremy_bowers_interview = (query_to_df("""
                                        SELECT interview.person_id, person.name, interview.transcript
                                        FROM interview
                                        JOIN person ON person.id = interview.person_id
                                        WHERE person.id = 67318
                                       """))
with pd.option_context('display.max_colwidth', None):
    print(jeremy_bowers_interview)

   person_id           name  \
0      67318  Jeremy Bowers   

                                                                                                                                                                                                                                           transcript  
0  I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n  


In [1350]:
who_hired_jeremy_bowers = query_to_df("""
                                            SELECT *
                                            FROM drivers_license
                                            WHERE car_make LIKE '%Tesla%' AND car_model LIKE '%S' AND hair_color = 'red' AND gender = 'female'
                                            """)
print(who_hired_jeremy_bowers)

       id  age  height eye_color hair_color  gender plate_number car_make  \
0  202298   68      66     green        red  female       500123    Tesla   
1  291182   65      66      blue        red  female       08CM64    Tesla   
2  918773   48      65     black        red  female       917UU3    Tesla   

  car_model  
0   Model S  
1   Model S  
2   Model S  


Following Jeremy Bowers transcript we found three possible females, who could've hired Jeremy Bowers. All three of them are in the <65", 67"> height range. I'm gonna check for more personal details next.

In [1351]:
personal_details = query_to_df("""
                                SELECT person.id, person.license_id, person.name, person.ssn, income.annual_income
                                FROM person
                                JOIN income ON person.ssn = income.ssn
                                WHERE license_id IN (202298, 291182, 918773)
                               """)
print(personal_details)
print('-------------------------------------------------------------------------')

facebook_event_checkin = query_to_df("""
                                      SELECT person.id, person.name
                                      FROM person, facebook_event_checkin
                                      WHERE person.id IN (78881, 99716) 
                                        AND person.id = facebook_event_checkin.person_id
                                        AND date >= 20171201 AND date < 20180101
                                        AND facebook_event_checkin.event_name LIKE '%SQL%Symphony%Concert%'
                                      GROUP BY person.id
                                      HAVING COUNT(date) = 3;
                                     """)
print(facebook_event_checkin)

#This is the last query needed to solve the mystery

      id  license_id              name        ssn  annual_income
0  78881      918773          Red Korb  961388910         278000
1  99716      202298  Miranda Priestly  987756388         310000
-------------------------------------------------------------------------
      id              name
0  99716  Miranda Priestly


# Final Verdict
By gathering all the information we eventually find out that the brain behind the murder in SQL City was the ***Miranda Priestly***.